<a href="https://colab.research.google.com/github/sayonaramagalhaes/Trabalho-Final---ML/blob/main/Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kagglehub

In [1]:
import os
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split, Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from tqdm import tqdm
from PIL import Image
import kagglehub
from sklearn.metrics import accuracy_score

#Baixar e carregar o dataset
path = kagglehub.dataset_download("shaunthesheep/microsoft-catsvsdogs-dataset")

# Mostrar o caminho do dataset
print("Path to dataset files:", path)

#Verificar a estrutura do diretório do dataset
pet_images_path = os.path.join(path, 'PetImages')

# Verificar se o diretório 'PetImages'
if os.path.exists(pet_images_path):
    print("Conteúdo do diretório PetImages:", os.listdir(pet_images_path))
else:
    print("O diretório PetImages não foi encontrado!")

#Definir o pré-processamento das imagens
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensiona para 224x224
    transforms.Grayscale(num_output_channels=3),  # Converte para 3 canais (RGB)
    transforms.ToTensor(),  # Converte para tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normaliza com os valores do ImageNet
])

# Função para verificar se a imagem é válida (ignorar imagens corrompidas)
def is_valid_image(image_path):
    try:
        with Image.open(image_path) as img:
            img.verify()  # Tenta verificar a imagem
        return True
    except (IOError, SyntaxError):
        return False

# Criar uma classe Dataset personalizada para carregar as imagens
class CustomDataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        return image, label

# Carregar as imagens e filtrar imagens válidas
img_paths = []
labels = []

for folder in os.listdir(pet_images_path):
    folder_path = os.path.join(pet_images_path, folder)
    if os.path.isdir(folder_path):
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            if is_valid_image(img_path):  # Verifica se a imagem é válida
                img_paths.append(img_path)
                label = 0 if folder == 'Cat' else 1
                labels.append(label)

# Criar dataset com imagens válidas
dataset = CustomDataset(img_paths, labels, transform)

# Dividir as imagens em treino (80%) e teste (20%)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, test_size])

# DataLoader para carregar os dados em batches
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

#Carregar o modelo AlexNet pré-treinado
model = models.alexnet(weights=models.AlexNet_Weights.IMAGENET1K_V1)

# Congelar todas as camadas convolucionais, pois queremos apenas treinar a última camada (classificação)
for param in model.parameters():
    param.requires_grad = False

# Substituir a camada final de classificação para 2 classes (Cachorro vs Gato)
model.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# Definir o dispositivo (GPU ou CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Colocar o modelo em modo de treinamento
model.train()

# Definir a função de perda (cross-entropy) e o otimizador (Adam)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.001)

# Treinamento do modelo com fine-tuning
epochs = 5
for epoch in range(epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    y_true = []
    y_pred = []

    for inputs, labels in tqdm(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)  # Move para o dispositivo (GPU ou CPU)

        optimizer.zero_grad()

        # Passar os dados pelo modelo
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        # Atualizar os parâmetros
        optimizer.step()

        # Estatísticas
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Armazenar rótulos e previsões para calcular as métricas
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

    # Calcular precisão
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    # Exibir as métricas a cada época
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {100 * correct / total:.2f}%")
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

#Avaliação do modelo
model.eval()
y_true = []
y_pred = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Calcular o relatório de classificação
report = classification_report(y_true, y_pred, target_names=['Cat', 'Dog'])

# Exibir o relatório de classificação
print("\nClassification Report (Test Set):")
print(report)

# Calcular Acurácia
accuracy = accuracy_score(y_true, y_pred)
print(f"Acurácia no conjunto de teste: {accuracy:.4f}")


Path to dataset files: /root/.cache/kagglehub/datasets/shaunthesheep/microsoft-catsvsdogs-dataset/versions/1
Conteúdo do diretório PetImages: ['Dog', 'Cat']


/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:01<00:00, 128MB/s]
100%|██████████| 625/625 [12:48<00:00,  1.23s/it]


Epoch 1/5, Loss: 0.1917, Accuracy: 92.67%
Precision: 0.9267, Recall: 0.9267, F1-Score: 0.9267


 75%|███████▍  | 468/625 [09:26<02:52,  1.10s/it]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 625/625 [12:39<00:00,  1.21s/it]


Epoch 2/5, Loss: 0.1804, Accuracy: 93.52%
Precision: 0.9352, Recall: 0.9352, F1-Score: 0.9352


...

Epoch 3/5, Loss: 0.1771, Accuracy: 93.95%
Precision: 0.9395, Recall: 0.9395, F1-Score: 0.9395


 25%|██▍       | 156/625 [03:10<09:21,  1.20s/it]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 625/625 [12:40<00:00,  1.22s/it]


Epoch 4/5, Loss: 0.1705, Accuracy: 94.10%
Precision: 0.9410, Recall: 0.9410, F1-Score: 0.9410


 79%|███████▊  | 492/625 [09:56<02:55,  1.32s/it]/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))
100%|██████████| 625/625 [12:37<00:00,  1.21s/it]


Epoch 5/5, Loss: 0.1718, Accuracy: 94.16%
Precision: 0.9416, Recall: 0.9416, F1-Score: 0.9416

Classification Report (Test Set):
              precision    recall  f1-score   support

         Cat       0.96      0.92      0.94      2466
         Dog       0.92      0.96      0.94      2534

    accuracy                           0.94      5000
   macro avg       0.94      0.94      0.94      5000
weighted avg       0.94      0.94      0.94      5000

Acurácia no conjunto de teste: 0.9392
